In [109]:
# import libraries
import urllib.request
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import numpy as np
import os
path = r"C:\Users\Quyen\OneDrive - GNS Science\Offline_work\11_Github\housepriceprediction"
chrome_path = r"C:\Users\Quyen\OneDrive - GNS Science\Offline_work\11_Github\chromedriver-win64\chromedriver.exe"
os.chdir(path)

In [ ]:
def get_html_data(url, driver):
    driver.get(url)
    time.sleep(np.random.lognormal(0,1))
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    return soup

In [119]:
# specify the url
urlpage = 'https://www.realestate.co.nz/residential/sale/otago/dunedin-city?by=oldest' 
print(urlpage)
# run firefox webdriver from executable path of your choice
service = Service(executable_path=chrome_path)
driver = webdriver.Chrome(service=service)
# get web page
driver.get(urlpage)
# execute script to scroll down the page
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
# sleep for 30s
time.sleep(3)
# driver.quit()

https://www.realestate.co.nz/residential/sale/otago/dunedin-city?by=oldest


In [127]:
#Search for page containers
page_container = driver.find_element(By.CSS_SELECTOR, "div[data-test='paginated-items']")
page_numbers = []
if page_container:
    all_links = page_container.find_elements(By.TAG_NAME, "a")
    for link in all_links:
        page_num = link.text.strip()
        page_numbers.append(page_num)
# Filter only numeric page numbers and convert to int
page_nums_int = [int(p) for p in page_numbers if p.isdigit()]
min_page = min(page_nums_int) if page_nums_int else None
max_page = max(page_nums_int) if page_nums_int else None
print("Min page:", min_page)
print("Max page:", max_page)

# Base url 
base_url =  'https://www.realestate.co.nz/residential/sale/otago/dunedin-city?by=oldest' 
# Creater master url 
master_url_dict = {}
master_rows = []
for page_num in range(min_page, max_page + 1):
    url = f"{base_url}&page={page_num}"
    print(f"Processing page: {url}")
    master_url_dict[page_num] = url


Min page: 1
Max page: 35
Processing page: https://www.realestate.co.nz/residential/sale/otago/dunedin-city?by=oldest&page=1
Processing page: https://www.realestate.co.nz/residential/sale/otago/dunedin-city?by=oldest&page=2
Processing page: https://www.realestate.co.nz/residential/sale/otago/dunedin-city?by=oldest&page=3
Processing page: https://www.realestate.co.nz/residential/sale/otago/dunedin-city?by=oldest&page=4
Processing page: https://www.realestate.co.nz/residential/sale/otago/dunedin-city?by=oldest&page=5
Processing page: https://www.realestate.co.nz/residential/sale/otago/dunedin-city?by=oldest&page=6
Processing page: https://www.realestate.co.nz/residential/sale/otago/dunedin-city?by=oldest&page=7
Processing page: https://www.realestate.co.nz/residential/sale/otago/dunedin-city?by=oldest&page=8
Processing page: https://www.realestate.co.nz/residential/sale/otago/dunedin-city?by=oldest&page=9
Processing page: https://www.realestate.co.nz/residential/sale/otago/dunedin-city?by

In [ ]:
# create empty array to store data
data = []
for page_num in range(min_page, max_page + 1):
    page_url = master_url_dict[page_num]
    # get web page
    driver.get(page_url)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
    soup = get_html_data(url, driver)
    results = driver.find_elements(By.CLASS_NAME, "tile--body")
    print(f'Page {page_num} - Number of results:', len(results))
    
    for result in results:
        product_name = result.text
        link = result.find_element(By.TAG_NAME, 'a')
        product_link = link.get_attribute("href")
        data.append({
            "page": page_num,
            "product": product_name,
            "link": product_link
        })
        print(len(data))
data_final = pd.DataFrame(data)
data_final.to_csv('HousepriceURLdunedin.csv')

Page 1 - Number of results: 19
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
Page 2 - Number of results: 19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
Page 3 - Number of results: 19
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
Page 4 - Number of results: 19
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76


KeyboardInterrupt: 

In [ ]:

data_df = pd.DataFrame(data)
data_df.head()

,product,link
0,,https://www.realestate.co.nz/42799557/resident...
1,,https://www.realestate.co.nz/42828623/resident...
2,,https://www.realestate.co.nz/42804784/resident...
3,,https://www.realestate.co.nz/42824903/resident...
4,,https://www.realestate.co.nz/42783392/resident...
5,,https://www.realestate.co.nz/42736218/resident...
6,,https://www.realestate.co.nz/42799281/resident...
7,,https://www.realestate.co.nz/42743508/resident...
8,,https://www.realestate.co.nz/42773358/resident...
9,,https://www.realestate.co.nz/42817162/resident...


In [33]:
#Go to each site to get the new data 
for index, row in data_df.iterrows():
     # access data using column names
     print(index, row['product'], row['link'])

0 Save this listing
OPEN HOME TOMORROW
1 Glenross Street, Glenross
Enquiries Over $539,000
3
1
House https://www.realestate.co.nz/42812047/residential/sale/1-glenross-street-glenross
1 Save this listing
OPEN HOME TOMORROW
16 Pinfold Place, Mosgiel
Negotiation
4
2
House https://www.realestate.co.nz/42820834/residential/sale/16-pinfold-place-mosgiel
2 Save this listing
OPEN HOME TOMORROW
27 Normanby Street, Saint Kilda
Deadline Sale
3
1
House https://www.realestate.co.nz/42826534/residential/sale/27-normanby-street-saint-kilda
3 10 Geelong Street, Waikouaiti
Enquiries Over $775,000
3 https://www.realestate.co.nz/42799557/residential/sale/10-geelong-street-waikouaiti
4  https://www.realestate.co.nz/42826535/residential/sale/1-copeland-street-brighton
5  https://www.realestate.co.nz/42829525/residential/sale/17-collins-street-waikouaiti
6  https://www.realestate.co.nz/42790427/residential/sale/248a-tirohanga-road-north-taieri
7  https://www.realestate.co.nz/42773372/residential/sale/928-ha

In [85]:
def get_html_data(url, driver):
    driver.get(url)
    time.sleep(np.random.lognormal(0,1))
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    return soup
def get_text_by_data_test(soup, tag, data_test):
    try:
        el = soup.find(tag, attrs={"data-test": data_test})
        return el.get_text(strip=True) if el else None
    except:
        return None

def get_capital_value_and_date(soup):
    container = soup.find("div", attrs={"data-test-reinz-valuation-badge": True})
    if not container:
        return {}

    updated_date = container.find("p").get_text(strip=True).replace("Updated", "").strip()
    capital_value = container.find("h4", attrs={"data-test-reinz-valuation-badge-value": True}).get_text(strip=True)

    return {
        "capital_value": capital_value,
        "updated_date": updated_date
    }

def get_estimated_values(soup):
    results = []
    sections = soup.find_all("div", attrs={"data-test-reinz-valuation-badge": True})
    for section in sections:
        try:
            confidence = section.find("p").get_text(strip=True)
            value = section.find("h4", attrs={"data-test-reinz-valuation-badge-value": True}).get_text(strip=True)
            results.append({"confidence_band": confidence, "estimated_value": value})
        except:
            results.append({"confidence_band": None, "estimated_value": None})
    return results

def capital_value_detail(soup):
    container = soup.find("div", {"data-test": "capital-valuation"})
    if not container:
        return {}

    capital_value = container.find("h4").get_text(strip=True)
    date_text = container.find("div", class_="opacity-50").get_text(" ", strip=True)
    valuation_date = date_text.replace("Valuation date:", "").strip()

    values = container.find_all("div", class_="font-semibold")
    land_value = values[0].get_text(strip=True) if len(values) > 0 else None
    improvement_value = values[1].get_text(strip=True) if len(values) > 1 else None

    return {
        "capital_value": capital_value,
        "valuation_date": valuation_date,
        "land_value": land_value,
        "improvement_value": improvement_value
    }

def get_feature(soup, feature_name):
    try:
        tag = soup.find("title", string=feature_name)
        if tag:
            return tag.find_parent("div", class_="flex items-center").find("span").get_text(strip=True)
        return np.nan
    except:
        return np.nan

all_listings = []

for index, row in data_df.iterrows():
    print(index, row['product'], row['link'])
    name = row['product']
    url = row['link']
    soup = get_html_data(url, driver)

    listing_info = {
        "product": name,
        "url": url,
        "title": get_text_by_data_test(soup, "h1", "listing-title"),
        "subtitle": get_text_by_data_test(soup, "h2", "listing-subtitle"),
        "description": get_text_by_data_test(soup, "div", "description-content__description"),
        "pricing_method": get_text_by_data_test(soup, "div", "listing-pricing-method"),
    }

    listing_info.update(get_capital_value_and_date(soup))

    confidence_list = get_estimated_values(soup)
    for i, entry in enumerate(confidence_list, 1):
        listing_info[f"confidence_band_{i}"] = entry.get("confidence_band")
        listing_info[f"estimated_value_{i}"] = entry.get("estimated_value")

    listing_info.update(capital_value_detail(soup))

    features = ["Bedroom", "Bathroom", "Floor area", "Land area", "Title type", "Garage"]
    feature_data = {f.lower().replace(" ", "_"): get_feature(soup, f) for f in features}
    listing_info.update(feature_data)

    all_listings.append(listing_info)

# Now all_listings is a list of dicts with combined data for all products
df_combined = pd.DataFrame(all_listings)


0 Save this listing
OPEN HOME TOMORROW
1 Glenross Street, Glenross
Enquiries Over $539,000
3
1
House https://www.realestate.co.nz/42812047/residential/sale/1-glenross-street-glenross
1 Save this listing
OPEN HOME TOMORROW
16 Pinfold Place, Mosgiel
Negotiation
4
2
House https://www.realestate.co.nz/42820834/residential/sale/16-pinfold-place-mosgiel
2 Save this listing
OPEN HOME TOMORROW
27 Normanby Street, Saint Kilda
Deadline Sale
3
1
House https://www.realestate.co.nz/42826534/residential/sale/27-normanby-street-saint-kilda
3 10 Geelong Street, Waikouaiti
Enquiries Over $775,000
3 https://www.realestate.co.nz/42799557/residential/sale/10-geelong-street-waikouaiti
4  https://www.realestate.co.nz/42826535/residential/sale/1-copeland-street-brighton
5  https://www.realestate.co.nz/42829525/residential/sale/17-collins-street-waikouaiti
6  https://www.realestate.co.nz/42790427/residential/sale/248a-tirohanga-road-north-taieri
7  https://www.realestate.co.nz/42773372/residential/sale/928-ha

In [87]:
df_combined.head()

,product,url,title,subtitle,description,pricing_method,capital_value,updated_date,confidence_band_1,estimated_value_1,...,estimated_value_4,valuation_date,land_value,improvement_value,bedroom,bathroom,floor_area,land_area,title_type,garage
0,Save this listing\nOPEN HOME TOMORROW\n1 Glenr...,https://www.realestate.co.nz/42812047/resident...,"1 Glenross Street, Glenross, Dunedin City",Affordable Glenross Gem with Modern Comforts,Discover this affordable opportunity in the so...,"dollarEnquiries Over $539,000","$510,000",Low,Low,$496K,...,$510K,1 Jul 2022,"$225,000","$285,000",3,1,NaN,470m2,NaN,NaN
1,Save this listing\nOPEN HOME TOMORROW\n16 Pinf...,https://www.realestate.co.nz/42820834/resident...,"16 Pinfold Place, Mosgiel, Dunedin City",Sun soaked elegance with sweeping Taieri views,You will be impressed with this beautifully de...,dollarNegotiation,"$1,090,000",Low,Low,$1.05M,...,$1.09M,1 Jul 2022,"$430,000","$660,000",4,2,192m2,1062m2,Freehold,2
2,Save this listing\nOPEN HOME TOMORROW\n27 Norm...,https://www.realestate.co.nz/42826534/resident...,"27 Normanby Street, Saint Kilda, Dunedin City",Calling all First Home Buyers and Investors,"Set in the sought-after suburb of St Kilda, th...",dollarDeadline Sale,"$570,000",Low,Low,$388K,...,$570K,1 Jul 2022,"$430,000","$140,000",3,1,NaN,466m2,NaN,NaN
3,"10 Geelong Street, Waikouaiti\nEnquiries Over ...",https://www.realestate.co.nz/42799557/resident...,"10 Geelong Street, Waikouaiti, Dunedin City",Dunedin to Coastal Living - 30 minutes away!,Built in 2021 and immaculately cared for ever ...,"dollarEnquiries Over $775,000","$840,000",Low,Low,$762K,...,$840K,1 Jul 2022,"$275,000","$565,000",3,2,146m2,1025m2,NaN,NaN
4,,https://www.realestate.co.nz/42826535/resident...,"1 Copeland Street, Brighton, Dunedin City","Wee Home, First Home, Last Home, Your Home","Cozy seaside gem, your dream home awaits! Pict...",dollarNegotiation,"$410,000",Low,Low,$325K,...,$410K,1 Jul 2022,"$255,000","$155,000",1,1,55m2,536m2,Freehold,NaN


In [89]:
df_combined.to_csv("Houseprice.csv")